<a href="https://colab.research.google.com/github/ladsong/if697-2020.2-data-science/blob/projeto1/Spotify_year_trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Coleta

### Kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle é o caminho onde o arquivo kaggle.json está presente do Google Drive
#Mudar o diretorio
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
!kaggle datasets download -d yamaerenay/spotify-dataset-19212020-160k-tracks

 96% 187M/195M [00:02<00:00, 99.8MB/s]
100% 195M/195M [00:02<00:00, 93.9MB/s]


In [ ]:
#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

Archive:  spotify-dataset-19212020-160k-tracks.zip
  inflating: artists.csv             
  inflating: data_by_artist_o.csv    
  inflating: data_by_genres_o.csv    
  inflating: data_by_year_o.csv      
  inflating: data_o.csv              
  inflating: dict_artists.json       
  inflating: tracks.csv              


In [ ]:
df_artists = pd.read_csv('artists.csv')
df_data_by_artist = pd.read_csv('data_by_artist_o.csv')
df_data_by_genres = pd.read_csv('data_by_genres_o.csv')
df_data_by_year = pd.read_csv('data_by_year_o.csv')
df_data = pd.read_csv('data_o.csv')
df_tracks = pd.read_csv('tracks.csv')